# Rectify Traces
This notebook demonstrates how to rectify a detector readout from a spectroscopic simulation using Scopesim. "Rectification" means the transformation of a spectral trace from the detector onto a rectangular pixel grid of wavelength and spatial position along the slit. Wavelength calibration and rectification are major tasks of the spectroscopic data-reduction pipeline. For convenience, Scopesim includes functionality to perform these tasks by reversing the *known* mapping that was used for the simulation, resulting in easily analysable 2D spectra that include all the noise and background components but neglect the uncertainties of a wavelength calibration as it would be performed during the reduction of real data. 
Rectification is demonstrated on a MICADO long-slit simulation.

In [ ]:
import numpy as np

from astropy import units as u
from astropy.wcs import WCS

from synphot import SourceSpectrum, Empirical1D
from scopesim_templates.micado import flatlamp
from scopesim_templates.micado.spectral_calibrations import line_list


In [ ]:
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline

In [ ]:
import scopesim as sim
# sim.bug_report() # display ScopeSim configuration

In [ ]:
# Edit this path if you have a custom install directory, otherwise comment it out.
# sim.rc.__config__["!SIM.file.local_packages_path"] = "inst_simulations/inst_pkgs/"

If you haven’t got the instrument packages yet, uncomment the following cell.

In [ ]:
#sim.download_packages(["MICADO", "ELT", "Armazones"])

## Creation of a source - spectral line lamp

As an example, we use a calibration lamp with equally spaced and equally strong emission lines, covering the full MICADO waveband. The line list is turned into a spectrum by placing a narrow Gaussian at each line position. To simulate the lamp, we use the `flatlamp` function and replace the default spectrum (a black body) by the line spectrum.

In [ ]:
lines = np.arange(0.8, 2.5, 0.01)

wave = np.linspace(0.8, 2.5, 4096)
flux = np.zeros_like(wave)
sigma = 0.00005
for line in lines:
    flux += 1e0 * np.exp(-(wave - line)**2 / (2 * sigma**2))

spec = SourceSpectrum(Empirical1D, points=wave*u.um, lookup_table=flux)

src_linelamp = flatlamp()
src_linelamp.fields[0].spectra[0] = spec     # NB: Do not try to set src_linelamp.spectra[0], this has no effect. 

We can also replace the test spectrum with an actual line-list

In [ ]:
# OPTIONAL: import line list from file, or comment out to keep the test spectrum
# llist = line_list(unit_flux=1e-12,
#                   dwave=0.01,
#                   smoothing_fwhm=0.03)

# src_linelamp.fields[0].spectra = llist.spectra

In [ ]:
plt.figure(figsize=(5,5))
plt.imshow(src_linelamp.fields[0].data)
plt.colorbar()

src_linelamp.spectra[0].plot()

## Simulation of an observation

We use MICADO in the long-slit spectroscopic mode.
Select `FILT` and `SLIT` as appropriate.
First iterage on flux levels before setting `USE_FULL_DETECTOR = True` for final simulation.

Possible combinations of filter and slit are:

    Spec_IJ, SPEC_3000x48
    Spec_IJ, SPEC_3000x16
    Spec_J, SPEC_15000x20
    Spec_HK, SPEC_3000x48
    Spec_HK, SPEC_1500020

In [ ]:
USE_FULL_DETECTOR = True
FILT = "Spec_IJ"
# FILT = "Spec_HK"
# FILT = "J"

# SLIT = "SPEC_3000x16"
SLIT = "SPEC_3000x48"
# SLIT = "SPEC_15000x20"

min_wavelen = {
    "Spec_IJ": 0.82,
    "Spec_HK": 1.49, 
    "J": 1.16,
    } # in microns

R = {
    "SPEC_3000x16": 10000,
    "SPEC_3000x48": 3300,
    "SPEC_15000x20": 8000,
}

# estimate spectral resolution in microns
def spec_res(slit, filter):
    return min_wavelen[filter] / R[slit]


spec_res_um = spec_res(SLIT, FILT)
print(f"{spec_res_um = }")

In [ ]:
# Set up ScopeSim parameters
cmds = sim.UserCommands(use_instrument="MICADO", set_modes=["SCAO", f"{SLIT}"])
cmds["!OBS.dit"] = 100
cmds["!OBS.ndit"] = 1
cmds["!OBS.filter_name_fw1"] = f"{FILT}"     # Spec_IJ, J, Spec_HK
cmds["!OBS.filter_name_fw2"] = "open"
cmds["!SIM.spectral.spectral_bin_width"] = spec_res_um
cmds["!DET.width"] = 4096
cmds["!DET.height"] = 4096

# turn off PSF interpolation to speed up simulations
cmds["!OBS.interp_psf"] = False

micado = sim.OpticalTrain(cmds)

We exclude atmospheric emission (and absorption) and the telescope optics as is appropriate for a calibration-lamp observation. As the source fills the slit homogeneously a PSF convolution should have no effect on the result. Excluding PSF convolution cuts down significantly on computation time.

In [ ]:
# Turn off the effects that are not needed (see below for more on this)
micado["skycalc_atmosphere"].include = False
micado["telescope_reflection"].include = False
micado["relay_psf"].include = False

micado["full_detector_array"].include = USE_FULL_DETECTOR
micado["detector_window"].include = not USE_FULL_DETECTOR

# For checking/debugging
# print(cmds.list_modes())
# micado.effects.pprint_all()
# micado["pupil_wheel"].meta
micado["filter_wheel_1"].current_filter.plot()

In [ ]:
# warning - this step takes ~12 minutes to run on the full detector
micado.observe(src_linelamp, update=True)

In [ ]:
# plot the image plane using a logarithmic intensity scale
plt.figure(figsize=(15,15))
plt.imshow(micado.image_planes[0].data, norm=LogNorm())
plt.colorbar()

In [ ]:
# check the spectrum of the last source used
micado._last_source.fields[0].spectra[0].plot()

Simulate detector readout - this projects the image plane onto the 3x3 detector focal-plane-array and adds readout noise.

In [ ]:
ro = micado.readout()[0]

In [ ]:
if USE_FULL_DETECTOR:
    # plot the 3x3 detector array
    fig, axs = plt.subplots(3, 3, figsize=(10,10), layout='constrained')
    for i, hdu in enumerate(ro[1:]):
        row = int(i/3)
        col = i%3
        if row == 1:
            col = 2 - col
        axs[row, col].imshow(hdu.data, origin="lower")
else:
    # plot the single detector window
    plt.figure(figsize=(10,10))
    plt.imshow(ro[1].data, origin="lower")
    plt.colorbar()

## Rectification of the spectrum

Rectification is necessary to arrive at a 2D spectrum with well-defined wavelength and spatial coordinates. The method to use is `rectify_traces` and belongs to the `SpectralTraceList` effect, which is accessible in the MICADO optical train as `"micado_spectral_traces"`. Currently, it is necessary to specify the spatial extent of the slit when calling the method. The long slit in MICADO has a length of 15 arcsec and extends from -5 arcsec to +10 arcsec. The short slit extens from -1.5 to +1.5 arcsec.

In [ ]:
tracelist = micado["micado_spectral_traces"]

In [ ]:
# warning - this step takes ~13 minutes to run
rectified = tracelist.rectify_traces(ro, -1.5, 1.5)
# skip primary header [0] and unused order 7_1 [6]
rectified = np.concatenate((rectified[1:6], rectified[7:]))

`rectified` is an HDU list with one extension for each spectral trace - for MICADO there are several. Each extension has a WCS that translates pixel coordinates into wavelength and position along the slit.

In [ ]:
# Plot the rectified traces
fig, axs = plt.subplots(len(rectified), figsize=(8,10), layout='constrained')

for i, trc in enumerate(rectified):
    wcs = WCS(trc)
    naxis1 = trc.header["NAXIS1"]
    naxis2 = trc.header["NAXIS2"]
    lam = (wcs.all_pix2world(np.arange(naxis1), 800, 0)[0] * u.Unit(wcs.wcs.cunit[0])).to(u.um).value
    xi = (wcs.all_pix2world(1000, np.arange(naxis2), 0)[1] * u.Unit(wcs.wcs.cunit[1])).to(u.arcsec).value
    axs[i].imshow(trc.data, origin="lower", extent=[lam[0], lam[-1], xi[0], xi[-1]])
    axs[i].set_aspect("auto")
    axs[i].set_ylabel(trc.header["EXTNAME"])

    # axs[i].set_ylabel(r"Spatial position along slit [arcsec]");

axs[-1].set_xlabel(r"Wavelength [$\mu$m]")


In [ ]:
# plot 1D extracted spectrum for each trace
fig, axs = plt.subplots(len(rectified), figsize=(8,10), layout='constrained')
for i, trc in enumerate(rectified):
    wcs = WCS(trc)
    naxis1 = trc.header["NAXIS1"]
    naxis2 = trc.header["NAXIS2"]
    lam = (wcs.all_pix2world(np.arange(naxis1), 800, 0)[0] * u.Unit(wcs.wcs.cunit[0])).to(u.um).value
    axs[i].plot(lam, trc.data[400], label="single row")
    axs[i].plot(lam, trc.data.mean(axis=0), label="average")
    axs[i].set_ylabel(trc.header["EXTNAME"])

axs[-1].set_xlabel(r"Wavelength [$\mu$m]")
plt.show()
    